# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**19/06/2020 - C12 S7**

# Redes Neuronales 

La redes neuronales aparecen bajo el nombre de **perceptron**, propuesta por [Rosenblatt en 1958](http://www.ling.upenn.edu/courses/cogs501/Rosenblatt1958.pdf). Este es un modelo de clasificación consistente en pesos $w$ y una función de output del tipo $f(w^t\mathbf{x})$ para el input $\mathbf{x}$. Si bien este modelo es en esencia muy similar a la regresión logística, el percetron implementa $f(\cdot)$ como una función de salto, que entrega el valor 1 para argumentos mayores que 0 y retorna 0 en otro caso. El problema de este modelo consiste en su poca expresibilidad, pues solo permite hacer separaciones lineales. 

Sobre el modelo anterior, se construye el perceptron multicapa, el cual consiste en múltiples modelos de perceptron intecomunicadas, cada uno de estos modelos se denota como *unidad* (*unit*) y se organizan en capas secuenciales conocidas como *capa input*, corresponde a un conjunto de perceptrones que operan de manera paralela sobre un vector input. El resultado de sus clasificaciones corresponde a un vector, donde cada componente se asocia un perceptron de la capa input. Las siguientes capas se denotan como *capas ocultas* y se contruyen de manera análoga a la capa input (o inicial), la primera capa oculta consiste en un conjunto de percetrones que operan sobre el output de la capa input. Sobre esta primera capa oculta pueden haber multiples capas ocultas que operan sobre el output de la capa oculta anterior. El proceso de generar una salida sobre la entrada de una capa anterior se denota *propagación hacia adelante* (*feedforward*) la propagación termina en una última capa denominada *capa output* que entrega el resultado final de la clasificación. 

Bajo el punto de vista del aprendizaje automático, las capas ocultas de un perceptron multicapa (red neuronal) generan abstracciones o características a partir de los datos sobre los cuales operan. 

Para trabajar con redes neuronales haremos uso de la librería **Pytorch**. Esta librería consiste en un conjunto de herramientas diseñadas para generar modelos basados en redes neuronales utilizando las capacidades de computo distribuido que ofrecen las GPU (unidades de procesamiento gráfico / tarjetas de video). Esto permite operaciones de vectorización aceleradas y distribuidas. Esta librería se importa como `torch`.

**Ejemplo**

Se implementa una red neuronal simple utilizando Pytorch. Para esto, se importa el módulo y se indica una semilla aleatoria.

In [ ]:
import torch

# Se asigna un valor de reproductibilidad
torch.manual_seed(6202)

Se carga el dataset sobre el cual trabajaremos, en este caso será un conjunto de datos de vinos. Este conjunto consta de 13 atributos continuos, cada vino posee un identificador dentro de 3 posibilidades, la idea es poder clasificar cada vino para predecir su identificador. 

In [ ]:
import pandas as pd

names = [
    'class','Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium',
    'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
    'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline'
]

wine_data = pd.read_csv(
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
    names=names)

wine_data.head()

Se procede a hacer una separación en entrenamiento y test, se hará es una codificación dummy para la variable de respuesta y se estandarizan las variables numéricas

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

response = ['class']
num_cols = wine_data.loc[:, 'Alcohol':].columns.values

data_transform = ColumnTransformer(
    transformers=[('normaliza', StandardScaler(),num_cols), 
                  ('codifica', OrdinalEncoder(), response)])

Para evitar fuga de información, se hace una separación en train y test para luego transformar los datos

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(wine_data, test_size=.2)

Se aplica el preprocesamiento sobre el conjunto train

In [ ]:
data = data_transform.fit_transform(data_train)

# Se obtienen las variables numericas y de respuesta

X_train = data[:,:-1].copy()
y_train = data[:,-1:].copy()

El perceptron multicapa a modelar consiste en 10 capas ocultas y su función de activación tiene la forma $f(x) = max(0,x)$, esta es una función conocida y se denota como ReLU (Rectified Linear Unit). Se definen los parámetros, para la función de activación se utiliza el módulo de redes neuronales de Pytorch, se puede acceder a este módulo por medio de `torch.nn`.

In [ ]:
input_dim, output_dim = (X_train.shape[1], 3)

# Equivalente al output de la capa input
input_dim_capas_ocultas = 5

f_activation = torch.nn.ReLU()

Para definir la primera capa, tenemos que tener en cuenta que buscamos una transformación de la forma $x \mapsto w^t x$ que luego es clasificada por medio de $f(w^t x)$. Esto quiere decir que los parámetros a entrenar serán los vectores de peso $w$ asociados a cada unidad (perceptron) de cada capa (input, ocultas y output). Pytorch ofrece una abstracción para capas formadas por perceptrones de la forma $f(w^t x)$. Esta abstracción es la clase `Linear` (observe que $x \mapsto w^t x$ es una transformación lineal), se procede a definir la capa input.




In [ ]:
from torch.nn import Linear

capa_input = Linear(in_features  = input_dim,
                    out_features = input_dim_capas_ocultas)

Luego se definen las capas ocultas, estas también corresponden a transformaciones lineales sobre sus capas predecesoras. En este caso, cada capa oculta tendrá dimensión 5 como input y output. Se define una capa oculta

In [ ]:
Linear(input_dim_capas_ocultas, input_dim_capas_ocultas)

La arquitectura de esta red es de naturaleza secuencial y corresponde al siguiente algoritmo:

1. Input: x de dimensión 13
2. Opera x en cada unidad de la capa lineal input, es decir, calcula $w_i^t x$ para $i = 1, \ldots, 5$ (dimensión de salida de la capa input).
3. Opera RelU($w_i^t x$) para $i = 1, \ldots, 5$, se obtiene un vector de dimensión 5. 

4. Para cada una de las 2 capas ocultas y de manera secuencial:
    1. Genera una transformación lineal sobre los outputs de la capa anterior.
    2. Aplica ReLU sobre las transformaciones lineales.
    3. En la última capa oculta el vector resultante se pasa a la capa output.

5. Se recibe el resultado de la última capa oculta, se transforma de manera lineal sobre 3 perceptrones para finalmente ser transformada por medio de una función softmax (dimensión del output es 3). 

Esto se reduce a: 
```
X  -> capa input -> ReLU() -> Co_1 -> ReLU() -> Co_2 -> ReLU() -> capa output -> predicción
```

Para programar el esquema anterior se hace uso de un diccionario ordenado `OrderedDict` de la librería `collections`. Este tipo de objetos opera de manera similar a las Pipelines de Scikit-learn, pues reciben un conjunto de tuplas del tipo `(identificador,objeto)`. Se procede a generar la arquitectura de la red:

In [ ]:
from collections import OrderedDict

n_capas_ocultas = 2

# capa input
input_ = ('capa input', capa_input)
relu = ('relu', f_activation)

steps = [input_, relu]

# capas ocultas
for i in range(n_capas_ocultas):
    
    capa = ('capa oculta_' + str(i),
         Linear(input_dim_capas_ocultas, input_dim_capas_ocultas))
    
    steps.extend([capa,relu])

# capa output
output = ('capa ouput', Linear(input_dim_capas_ocultas, output_dim))

steps.extend([(output),('Softmax',torch.nn.Softmax(dim=1))])

# Se utiliza la estructura de diccionario ordenado
steps = OrderedDict(steps)

Luego, cuando ya se posee la arquitectura, se inicializa un objeto `Sequential` del módulo `nn`, este objeto permite modelar una red neuronal multicapa recibiendo como input los componentes de la arquitectura de manera ordenada.

In [ ]:
from torch.nn import Sequential

#MLP -> multi layer perceptron
mlp = Sequential(steps)

El resultado entregado por la capa output corresponde a un vector de tres dimensiones, se asigna la clase predicha a aquella componente con el mayor valor. Luego de definir la red, es necesario definir un criterio de optimalidad (función de perdida), en este caso se utiliza la entropía cruzada. Este criterio permite comparar dos distribuciones de probabilidad $q$ (aproximación) y $p$ (real) en términos de la diferencia de información esperada (en bits por ejemplo) al utilizar la distribución $q$ para describir un eventos codificados, optimizados para $p$. Dado que se trabaja en un problema de clasificación (supervisado) se conoce la distribución real $p$ para una etiqueta  (ej: etiqueta (1,0,0), distribución (100%, 0, 0) ), por otra parte, la distribución aproximada viene dada por nuestro modelo. La entropía cruzada entre $p$ y $q$ se expresa según:
$$
H(p, q)=-\sum_{x \in \mathcal{X}} p(x) \log q(x)
$$

Se implementa mediante:

In [ ]:
criterio = torch.nn.CrossEntropyLoss()

Se debe seleccionar el optimizador a utilizar, en este caso será **descenso de gradiente estocástico** (SGD, se estudiará con más detenimiento posteriormente). Este optimizador tiene un parámetro asociado a la proporción de aprendizaje (*learning rate*) y momentum. Se inicializa entregando dichos parámetros y los coeficientes sobre los que opera, en este caso, los parámetros de la red `mpl` a los cuales se acceede por medio del método `.parameters()`

In [ ]:
optimizador = torch.optim.SGD(mlp.parameters(), lr=0.05, momentum=0.1)

Finalmente, se puede entrenar la red definida, para ello se define una cantidad de *épocas* (*epochs*), esto se refiere a la cantidad de veces que se entrena utilizando el conjunto de entrenamiento. Este proceso tiene el siguiente orden:

1. Genera un conjunto de inputs en un formato compatible.
2. En cada época:
    1. Inicializa los gradientes asociados al optimizador, esto evita que se acumulen gradientes entre épocas.
    2. Se opera sobre los inputs para obtener las predicciones.
    3. Se calcula el error de predicción.
    4. Se propagan los valores de la función de perdida para ajustar los pesos de la red y mejorar predicciones. Este proceso se denomina como *propagación hacia atrás* (*backpropagation*).
    5. Se pasa a la siguiente época.
    
Se implementa el esquema anterior:

In [ ]:
#Paso A
datos_input = torch.autograd.Variable(torch.Tensor(X_train))
labels = torch.autograd.Variable(torch.Tensor(y_train.reshape([-1,])).long())

In [ ]:
epocas = 1000
for ep in range(epocas):
    #Paso B
    optimizador.zero_grad()
    for i in range(20):
        #Paso C
        out = mlp(datos_input)
        #out.requires_grad_(True)

        #Paso D 
        loss = criterio(out, labels)
        
        #Paso E
        loss.backward() 
        optimizador.step()
    
    if ((ep+1) % 100) == 0 : print('Epoca: ', ep+1, 'Loss: ', loss.data)

Se estudia el rendimiento en train y test

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

dt_test = data_transform.transform(data_test)

X_test = dt_test[:, :-1]
y_test = dt_test[:, -1:]

f = lambda x: torch.argmax(mlp(x), dim=1)

#Train error
datos_input = torch.Tensor(X_train).float()
preds_train = f(datos_input)

print('Reporte train : \n',
      classification_report(y_train.reshape([-1,]), preds_train))

In [ ]:
# Test Error

datos_input = torch.Tensor(X_test).float()
preds_test = f(datos_input)

print('Reporte test : \n', 
      classification_report(y_test.reshape([-1,]), preds_test))

se puede decir que el clasificador entrenado fue un éxito.

Las redes neuronales pueden ser descritas como un modelo matemático de procesamiento de información. En general, una red neuronal puede considerarse como un sistema con las siguientes características:

1. El procesamiento de la información ocurre en unidades llamadas neuronas.
2. Las neuronas están conectadas e intercambian información (o señales) por medio de sus conexiones.
3. Las conexiones entre neuronas pueden ser fuertes o débiles, dependiendo de como se procesa la información.
4. Cada neurona tiene un estado interno determinado por todas las conexiones que posee.
5. Cada neurina tiene una función de activación que opera sobre su estado, esta función determina la información que se comparte a otras neuronas.

En términos operativos, una red neuronal posee una **arquitectura** que describe el conjunto de conexiones entre neurona y un proceso de **aprendizaje** asociado, que describe el proceso entrenamiento.

Las **neuronas** por tanto, pueden ser definidas por medio de la siguiente relación:
$$
y=f\left(\sum_{i} x_{i} w_{i}+b\right)
$$

Acá se hace el calculó $w^t x + b = \sum_i x_i w_i + b$ sobre los inputs $x_i$ y los pesos $w_i$. Estos últimos, son valores numéricos que representan las conexiones entre neuronas, el peso $b$ se denomina *bias*. Luego se calcula el resultado de aplicar la función de activación $f(\cdot)$. Existen distintos tipos de funciones de activación dentro de estas se pueden nombrar:

* $f(x)=x$ la función identidad.
* $f(x)=\left\{\begin{array}{l}1 \text { if } x \geq 0 \\ 0 \text { if } x<0\end{array}\right.$ la función de activación de umbral.
* $f(x)=\frac{1}{1+\exp (-x)}$ la función sigmoide logistica, es una de las más utilizadas.
* $f(x) =\frac{1-\exp (-x)}{1+\exp (-x)}$ la función sigmoide bipolar, esta corresponde a una sigmoide escalada a $(-1,1)$.
* $f(x) = \frac{1-\exp (-2 x)}{1+\exp (-2 x)}$ la tangente hiperbólica. 

* $f(x)=\left\{\begin{array}{l}x\text { if } x \geq 0 \\ 0 \text { if } x<0\end{array}\right.$ La función de activación ReLU.

**Ejercicio**

1. Existen variantes de la función de activación ReLU, investigue al menos 3 y compare sus diferencias.

Por su parte, las **capas de una red** conforman su arquitectura al agrupar conjuntos de neuronas. Acá se puede distinguir la *capa input* que representa las condiciones iniciales del dataset. La *capa output* puede tener una neurona (ej: problemas de regresión) o más (como en el ejemplo anterior). Si hay capas entre el input y el output, estas se denominan *capas ocultas*. En general las capas pueden estar completamente conectadas (como en el ejemplo implementado) o pueden faltar conexiones entre neuronas, esto se puede representar por los valores de los pesos, por ejemplo, el si denotamos $w_{ij}^{(k,k+1)}$ como los pesos entre el output de la neurona $i$ en la capa $k$ con el input de la neurona $j$ de la capa $k+1$, un peso $w_{ij}^{(k,k+1)} = 0$ implica que la neurona $j$ esima de la capa $k+1$ no posee interacción con la neurona $i$ esima de la capa $k$. 

**Ejercicios**

1. El esquema presentado en el ejemplo anterior implica que las capas de una red deben conectarse de manera secuencial. En general esto no tiene porque ser cierto, en efecto, las conexiones entre capas pueden ser modeladas por medio de grafos *acíclicos dirigidos*. ¿Por qué es necesario utilizar grafos de este tipo? 

2. El teorema de aproximación universal para redes neuronales dice que toda función continua en un compacto de $\mathbb{R}^n$ puede ser aproximada por una red neuronal con al menos una capa oculta. Para tener una intuición sobre esto, observe que es posible aproximar funciones continuas por medio de sumas ponderadas de *funciones boxcar* o *cajón*, estás, para una altura $A$ y base $(b-a)$ , $b >a$ se puede escribir como $\operatorname{boxcar}(x)=A(H(x-a)-H(x-b))$, donde:
$$
H(x)=\left\{\begin{array}{ll}0, & x<0 \\ 1, & x\geq 0\end{array}\right.
$$ 

Implemente una función boxcar utilizando una red neuronal con inputs en $x \in \mathbb{R}$, una capa oculta de dos neuronas, cada una con función de activación $f(x) = 1 /(1+\exp (-x))$, la red genera un output $y \in \mathbb{R}$.

El proceso de aprendizaje en redes pasa por definir un **esquema de entrenamiento**. Esto hace referencia a encontrar un conjunto de conexiones óptimas entre las las neuronas de la arquitectura para generar predicciones. Estas conexiones optimas se representan por los pesos de la red en cada capa y se entrenan en función de algún esquema de optimización (Descenso de gradiente por ejemplo).

Observemos por ejemplo una regresión lineal, en este caso la red consta de una capa oculta, un output unidimensional y la función de activiación identidad. Es posible entrenar una regresión lineal utilizando descenso de gradiente, para ello se define el función a minimizar $J$ como:
$$
J = \frac{1}{n} \sum_{i=0}^{n}\left(y_{i}-t_{i}\right)^{2}=\frac{1}{n} \sum_{i=0}^{n}\left(w^t x_{i} - {i}\right)^{2}
$$

En este caso $y_i$ representa el output del modelo lineal. El funcional a minimizar es el error cuadrátrico medio. El esquema de descenso de gradiente para este modelo pasa a ser:
$$
w \rightarrow w-\lambda \nabla(J(w))
$$

El cual se detiene al llegar a cierto umbral para $J$ o luego de cierto número de iteraciones. 

**Ejercicios**

1. Encuentre una expresión exacta para $\nabla(J(w))$ e implemente un esquema de descenso de gradiente para la regresión lineal. ¿Qué nombre se asocia al parámetro $\lambda$ en esquema $w \rightarrow w-\lambda \nabla(J(w))$?¿Cuál es su función?

2. El esquema de regresión logística corresponde a una red neuronal idéntica a la utilizada para regresión lineal, con la salvedad de que posee una función de activación sigmoide logística. Calcule un esquema de descenso de gradiente para este tipo de red. 

Se debe observar que al implementar el esquema anterior, se produce una acumulación de error en todo en conjunto de entrenamiento. En conjunto de datos grandes esto no es práctico. Una solución es implementar *Descenso de Gradiente Estocástico*, este algoritmo consiste en aproximar los pesos del esquema anterior sobre un subconjunto de observaciones del conjunto de entrenamiento, este subconjunto se denota como **mini -batch**.

Las redes de una capa oculta (regresión logística y lineal) poseen reglas sencillas de actualización según la función de activación, con esto se pueden actualizar los pesos de cada conexión utilizando el error de predicción como referencia. En redes con múltiples capas sólo se puede aplicar esta técnica a las neuronas de la última capa oculta, esto pues, no se tienen valores reales (*target*) con los cuales comparar los resultados de las capas ocultas intermedias. En este caso, lo que se hace es calcula el error de la ultima capa oculta para luego hacer una **estimación** del error cometido en la capa oculta anterior, este proceso se hace de manera recurrente hasta llegar a la capa inicial. Tal proceso de denota como *propagación hacia atrás* (**backpropagation**). Este algoritmo es la base del aprendizaje con redes neuronales. Para comprenderlo es necesario definir los siguientes conceptos:


1. Se define $w_ij$ como el peso entre la neurona $i$ esima de la capa $l$ con la neurona $j$ esima de la capa $l+1$. Es decir, el subindice izquierdo hace referencia a una capa anterior a la que neurona que representa el subindice derecho. Al convención $l$ hace referncia a cualquier capa oculta de la red.

2. Se utiliza $y$ para denotar inputs y outputs entre capas.  Es decir, $y_i$ es el input de la capa $l+1$ pero también el output de la capa $l$. 

3. El funcional de pérdida se denota como $J$, el valor de activación $w^t x$ se denota como $a$.

4. Si $a_j$ es una función de activación con pesos $w_j$, $y_j$ es una función de $a_j$ y $J$ es función de $y_j$. Según la regla de la cadena se tiene así:
$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$

5. Como se sabe que $\frac{\partial a_{j}}{\partial w_{i, j}}=y_{i}$, se tiene:

$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} y_{i}
$$

6. Para las capas anteriores se cumple la misma formula (por notación):$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$
Aunque se tengan muchas capas, siempre es posible concentrarse en pares de capas consecutivas, por que se logra siempre tener una capa input y una capa output. Como se sabe que $\frac{\partial a_{j}}{\partial w_{i, j}}=y_{i}$, es decir, la derivada parcial del valor de  activación de la neurona $j$-ésima en la capa $l+1$ con respecto al peso de la conexión del output $y_j$ entre las capas $l$ y $l+1$ con la neurona $i$-ésima, es justamente el valor del output sin ser ponderado por el peso de la conexión. Más aún, $\frac{\partial y_{j}}{\partial a_{j}}$ es la derivada de la función de activación que se puede calcular. Entonces, en la expresión obtenida por la regla de la cadena, sólo hace falta calcular $\frac{\partial J}{\partial y_{j}}$, la derivada del error con respecto a la función de activación en la segunda capa. Con esto, se pueden calcular todas las derivadas partiendo desde la última capa hacia atrás pues:
    * Se puede calcular esta derivada para la última capa
    * Se tiene la formula que permite calcula la derivdada de una capa, teniendo calculada la derivada de la capa siguiente. 

**Ejemplo**

En la siguiente ecuación, $y_i$ es el output de la primera capa, $y_j$ el ouput de la segunda capa. Aplicando la regla de la cadena se tiene:
$$
\frac{\partial J}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial y_{i}}
$$

Se puede calcular $\frac{\partial y_{j}}{\partial a_{j}}$ y $\frac{\partial a_{j}}{\partial y_{i}}=w_{i, j}$ , si $\frac{\partial J}{\partial y_{j}}$ es conocido (gradiente en la capa segunda), se puede calcular $\frac{\partial J}{\partial y_{i}}$ (gradiente en la capa primera). En resumen, si se tienen 3 capas conectadas de manera secuencia según los inputs (outputs) $y_i \rightarrow y_j \rightarrow y_k$ entonces se pueden aplicar las ecuaciones:

$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$

y 

$$
\frac{\partial J}{\partial y_{j}}=\sum_{k} \frac{\partial J}{\partial y_{k}} \frac{\partial y_{k}}{\partial y_{j}}
$$

Para calcular las derivadas del costo en cada capa. 

Al definir $\delta_{j}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}}$, se tiene que $\delta_j$ representa la variación de costo con respecto a cada valor de activación. Con esto se puede reescribir:

$$
\frac{\partial J}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial y_{i}}=\sum_{j} \delta_{j} w_{i, j}
$$

Con lo que $\delta_{i}=\left(\sum_{j} \delta_{j} w_{i, j}\right) \frac{\partial y_{i}}{\partial a_{i}}$. Esto se traduce en poder calcular la variación para la siguiente capa:

$$\delta_{j}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}}$$

$$\delta_{i}=\left(\sum_{j} \delta_{j} w_{i, j}\right) \frac{\partial y_{i}}{\partial a_{i}}$$

Combinando las ecuaciones:

$$
\frac{\partial J}{\partial w_{i, j}}=\delta_{j} \frac{\partial a_{j}}{\partial w_{i, j}}=\delta_{j} y_{i}
$$

Con lo que la regla de actualizació para los pesos de cada capa pasa a ser: 

$$
w_{i, j} \rightarrow w_{i, j}-\eta \delta_{j} y_{i}
$$

En Pytorch es posible implementar reglas de actualización basadas en propagación hacia atrás de manera automática. Para ello, es necesario comprender los tipos de datos asociados a esta librería. 

En primer lugar, se tienen los **tensores**, estos son arreglos n dimensionales, soportan almacenamiento en GPU. Poseen el atributo booleano `.requires_grad`, si es verdadero, un objeto tipo `Tensor` generará un grafo en cual se registra cada operación aplicada sobre él. Este grafo es utilizado para calcular gradientes de manera automática utilizando y se denota como grafo dinámico de computación DCG.

**Ejemplo**

Se define un tensor en Pytorch que hace uso del DCG para registrar operaciones y calcular gradientes. Pytorch se mimetiza con NumPy en cuanto a la definición de sus métodos y compatibilidad cruzada, por ejemplo, para definir un arreglo de 2x2 que requiera del uso de DCG se puede utilizar la función `randn` (análoga a la de NumPy)

In [ ]:
x = torch.randn(2, 2, requires_grad = True)
x

Se pueden importar arreglos de numpy de manera bastante sencilla

In [ ]:
x = np.random.randn(2,2)
x = torch.from_numpy(x)
x

De la misma forma se puede activar el uso de gradientes en objetos importados desde NumPy

In [ ]:
x.requires_grad_(True)

Como observación, se debe tener en cuenta que sólo es posible calcular gradientes para tensores de punto flotante. 

Por otra parte, la clase `Autograd` es el motor sobre el cual se calculan las derivadas. Esta clase registra todas las operaciones en un grafo acíclico dirigido. . Las hojas de este grafo son los tensores input y las raices son los tensores output. Los gradientes se calculan recorriendo el grafo desde la raiz a la hoja, multiplicando cada gradiente según la regla de la cadena. 

**Ejemplo**

Se crea un grafo usando tensores que requieren gradiente.

In [ ]:
x = torch.tensor(5.0, requires_grad = True)
y = torch.tensor(1.0)
z = x * y + 1

# Componentes del grafo
for i, name in zip([x, y, z], "XYZ"):
    
    
    print(f"{name}\ndata: {i.data}\nrequires_grad: {i.requires_grad}\n\
grad: {i.grad}\ngrad_fn: {i.grad_fn}\nis_leaf: {i.is_leaf}\n")

Para hacer operaciones sin registrar gradientes se utiliza el context manager `torch,no_grad()`.

In [ ]:
x = torch.tensor(25.0, requires_grad=True)
y = x * 2

print('Se requiere gradiente en y:', y.requires_grad)

with torch.no_grad():
    y = x * 2
    print('Se requiere gradiente en y (context manager): ', y.requires_grad)

El método `.backward()` calcula los gradientes al aplicarse sobre un **valor escalar** (tensor unitario) esto se recorriendo el grafo generado por las operaciones que le dan origen a tal valor. Los gradientes calculados se guardan en el atributo `.grad` de cada nodo hoja. 

**Ejemplo**

Se estudia el gradiente de una función 

In [ ]:
# Se genera el grafo de manera dinamica en cada operacion
x = torch.tensor(5.0, requires_grad = True)
z = x ** 3

# Se calculan los gradientes
z.backward() 

En el caso anterior, se tiene la función $z = x^3$ cuyo gradiente con respecto a $x$ es $3x^2$ que evaluado en $x=5$ pasa a ser $75$. Se confirma lo anterior

In [ ]:
print(x.grad.data)

Cuando se llama `z.backward()` se pasa como argumento el tensor unitario `torch.tensor(5.0)`, es decir, se calcula `z.backward(torch.tensor(5.0))`, en este caso, dicho argumento es el *gradiente externo* utilizado para terminar las multiplicaciones asociadas a la regla de la cadena. Por ejemplo, si se tienen los tensores :

In [ ]:
x = torch.tensor([2.0, 4.0, 8.0], requires_grad = True)
y = torch.tensor([1.0 , 3.0 , 5.0], requires_grad = True)

z = x*y
z

In [ ]:
z.backward(torch.tensor([1.0,1.0,1.0]))
x.grad

Entonces, para calcular los gradientes de `z` con respecto a `x` o a `y`, se debe entregar un gradiente externo de dimensión 3. Este tensor actúa como los pesos de un gradiente ponderado. Es decir, corresponde a multiplicar un vector de pesos por la matriz jacobiana. En término generales, la clase Autograd permite calcular productos de la matriz jacobiana con vectores de gradiente de la forma:

$$
J \cdot v=\left(\begin{array}{ccc}\frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}\end{array}\right)\left(\begin{array}{c}\frac{\partial l}{\partial y_{1}} \\ \vdots \\ \frac{\partial l}{\partial y_{m}}\end{array}\right)=\left(\begin{array}{c}\frac{\partial l}{\partial x_{1}} \\ \vdots \\ \frac{\partial l}{\partial x_{n}}\end{array}\right)
$$